<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>

# Fusionner des données

## Exercices

Cet exercice vient en deux versions, une de niveau basique et une de niveau intermédiaire.

La version basique est une application de la technique d'indexation que l'on a vue dans le complément "Gérer des enregistrements". On peut très bien faire les deux versions dans l'ordre, une fois qu'on a fait la version basique on est en principe un peu plus avancé pour aborder la version intermédiaire.

### Contexte

Nous allons commencer à utiliser des données un peu plus réalistes. Il s'agit de données obtenues auprès de [MarineTraffic](https://www.marinetraffic.com) - et légèrement simplifiées pour les besoins de l'exercice. Ce site expose les coordonnées géographiques de bateaux observées en mer au travers d'un réseau de collecte de type *crowdsourcing*.

De manière à optimiser le volume de données à transférer, l'API de MarineTraffic offre deux modes pour obtenir les données :

* **mode étendu** : chaque mesure (bateau x position x temps) est accompagnée de tous les détails du bateau (`id`, nom, pays de rattachement, etc.) ;
* **mode abrégé** : chaque mesure est uniquement attachée à l'`id` du bateau.

En effet, chaque bateau possède un identifiant unique qui est un entier, que l'on note  `id`.

### Chargement des données

Commençons par charger les données de l'exercice :

In [1]:
from corrections.exo_marine_dict import extended, abbreviated

### Format des données

Le format de ces données est relativement simple, il s'agit dans les deux cas d'une liste d'entrées - une par bateau.

Chaque entrée à son tour est une liste qui contient :

    mode étendu: [id, latitude, longitude, date_heure, nom_bateau, code_pays, ...]
    mode abrégé: [id, latitude, longitude, date_heure]

sachant que les entrées après le code pays dans le format étendu ne nous intéressent pas pour cet exercice.

In [2]:
# une entrée étendue est une liste qui ressemble à ceci
sample_extended_entry = extended[3]
print(sample_extended_entry)

[255801560, 49.3815, -4.412167, '2013-10-08T21:51:00', 'AUTOPRIDE', 'PT', '', 'ZEEBRUGGE']


In [3]:
# une entrée abrégée ressemble à ceci
sample_abbreviated_entry = abbreviated[0]
print(sample_abbreviated_entry)

[227254910, 49.91799, -5.315172, '2013-10-08T22:59:00']


On précise également que les deux listes `extended` et `abbreviated` :

* possèdent exactement **le même nombre** d'entrées ;
* et correspondent **aux mêmes bateaux** ;
* mais naturellement **à des moments différents** ;
* et **pas forcément dans le même ordre**.

*******

### Exercice - niveau basique

In [4]:
# chargement de l'exercice
from corrections.exo_marine_dict import exo_index

##### But de l'exercice

On vous demande d'écrire une fonction `index` qui calcule, à partir de la liste des données étendues, un dictionnaire qui est :

* indexé par l'`id` de chaque bateau ;
* et qui a pour valeur la liste qui décrit le bateau correspondant.

De manière plus imagée, si :

```Python
extended = [ bateau1, bateau2, ... ]
```

Et si :

```Python
bateau1 = [ id1, latitude, ... ]
```

On doit obtenir comme résultat de `index` un dictionnaire :

```Python
{
    id1 -> [ id_bateau1, latitude, ... ],
    id2 ...
}
```

Bref, on veut pouvoir retrouver les différents éléments de la liste `extended` par accès direct, en ne faisant qu'une seule recherche dans l'index.

In [5]:
# le résultat attendu
result_index = exo_index.resultat(extended)

# on en profite pour illustrer le module pprint
from pprint import pprint

# à quoi ressemble le résultat pour un bateau au hasard
for key, value in result_index.items():
    print("==== clé")
    pprint(key)
    print("==== valeur")
    pprint(value)
    break

==== clé
992271012
==== valeur
[992271012, 47.64744, -3.509282, '2013-10-08T21:50:00', 'PENMEN', 'FR', '', '']


Remarquez ci-dessus l'utilisation d'un utilitaire parfois pratique : le [module `pprint` pour pretty-printer](https://docs.python.org/3/library/pprint.html).

##### Votre code

In [8]:
a = [1,2,3,4,5]
a[1:]

[2, 3, 4, 5]

In [15]:
def index(extended):
    bateau = dict()
    for content in extended:
        bateau.update({content[0]: content[:]})
    return bateau

##### Validation

In [16]:
exo_index.correction(index, abbreviated)

Appel,Attendu,Obtenu,
"index( [ [ 227254910, 49.91799, -5.315172, '2013-10-08T22:59:00'], [ 255801560, 49.25383, -4.784833, '2013-10-08T22:59:00'], [ 992271012, 47.64748, -3.509307, '2013-10-08T22:56:00'], [ 227844000, 47.13057, -3.126982, '2013-10-08T22:58:00']])","{ 227254910: [ 227254910, 49.91799, -5.315172, '2013-10-08T22:59:00'], 227844000: [ 227844000, 47.13057, -3.126982, '2013-10-08T22:58:00'], 255801560: [ 255801560, 49.25383, -4.784833, '2013-10-08T22:59:00'], 992271012: [ 992271012, 47.64748, -3.509307, '2013-10-08T22:56:00']}","{ 227254910: [ 227254910, 49.91799, -5.315172, '2013-10-08T22:59:00'], 227844000: [ 227844000, 47.13057, -3.126982, '2013-10-08T22:58:00'], 255801560: [ 255801560, 49.25383, -4.784833, '2013-10-08T22:59:00'], 992271012: [ 992271012, 47.64748, -3.509307, '2013-10-08T22:56:00']}",OK


Vous remarquerez d'ailleurs que la seule chose que l'on utilise dans cet exercice, c'est que l'id des bateaux arrive en première position (dans la liste qui matérialise le bateau), aussi votre code doit marcher à l'identique avec les bateaux étendus :

In [17]:
exo_index.correction(index, extended)

Appel,Attendu,Obtenu,
"index( [ [ 992271012, 47.64744, -3.509282, '2013-10-08T21:50:00', 'PENMEN', 'FR', '', ''], [ 227254910, 49.94479, -5.137455, '2013-10-08T21:51:00', 'LAURELINE', 'FR', '', 'CHERBOURG'], [ 227844000, 47.23206, -2.913887, '2013-10-08T21:51:00', 'NEPTUNE III', 'FR', '', ''], [ 255801560, 49.3815, -4.412167, '2013-10-08T21:51:00', 'AUTOPRIDE', 'PT', '', 'ZEEBRUGGE']])","{ 227254910: [ 227254910, 49.94479, -5.137455, '2013-10-08T21:51:00', 'LAURELINE', 'FR', '', 'CHERBOURG'], 227844000: [ 227844000, 47.23206, -2.913887, '2013-10-08T21:51:00', 'NEPTUNE ' 'III', 'FR', '', ''], 255801560: [ 255801560, 49.3815, -4.412167, '2013-10-08T21:51:00', 'AUTOPRIDE', 'PT', '', 'ZEEBRUGGE'], 992271012: [ 992271012, 47.64744, -3.509282, '2013-10-08T21:50:00', 'PENMEN', 'FR', '', '']}","{ 227254910: [ 227254910, 49.94479, -5.137455, '2013-10-08T21:51:00', 'LAURELINE', 'FR', '', 'CHERBOURG'], 227844000: [ 227844000, 47.23206, -2.913887, '2013-10-08T21:51:00', 'NEPTUNE ' 'III', 'FR', '', ''], 255801560: [ 255801560, 49.3815, -4.412167, '2013-10-08T21:51:00', 'AUTOPRIDE', 'PT', '', 'ZEEBRUGGE'], 992271012: [ 992271012, 47.64744, -3.509282, '2013-10-08T21:50:00', 'PENMEN', 'FR', '', '']}",OK


### Exercice - niveau intermédiaire

In [18]:
# chargement de l'exercice
from corrections.exo_marine_dict import exo_merge

##### But de l'exercice

On vous demande d'écrire une fonction `merge` qui fasse une consolidation des données, de façon à obtenir en sortie un dictionnaire :

```Python
id -> [nom_bateau, code_pays, position_etendu, position_abrege]
```

dans lequel les deux objets `position` sont tous les deux des tuples de la forme :

```Python
(latitude, longitude, date_heure)
```

Voici par exemple un couple clé-valeur dans le résultat attendu :

In [19]:
# le résultat attendu
result_merge = exo_merge.resultat(extended, abbreviated)

# à quoi ressemble le résultat pour un bateau au hasard
from pprint import pprint
for key_value in result_merge.items():
    pprint(key_value)
    break

(992271012,
 ['PENMEN',
  'FR',
  (47.64744, -3.509282, '2013-10-08T21:50:00'),
  (47.64748, -3.509307, '2013-10-08T22:56:00')])


##### Votre code

In [46]:
def merge(extended, abbreviated):
    dico = dict()
    for elt in extended:
        id_bat = elt[0]
        data = tuple(elt[1:4])
        nom_bateau = elt[4]
        code_pays = elt[5]
        dico[id_bat]=[nom_bateau, code_pays, data]
    
    for elt in abbreviated:
        id_bat = elt[0]
        data = elt[1:]
        dico[id_bat].append(tuple(data))
        
    return(dico)

##### Validation

In [47]:
exo_merge.correction(merge, extended, abbreviated)

Appel,Attendu,Obtenu,
"merge( [ [ 992271012, 47.64744, -3.509282, '2013-10-08T21:50:00', 'PENMEN', 'FR', '', ''], [ 227254910, 49.94479, -5.137455, '2013-10-08T21:51:00', 'LAURELINE', 'FR', '', 'CHERBOURG'], [ 227844000, 47.23206, -2.913887, '2013-10-08T21:51:00', 'NEPTUNE III', 'FR', '', ''], [ 255801560, 49.3815, -4.412167, '2013-10-08T21:51:00', 'AUTOPRIDE', 'PT', '', 'ZEEBRUGGE']], [ [ 227254910, 49.91799, -5.315172, '2013-10-08T22:59:00'], [ 255801560, 49.25383, -4.784833, '2013-10-08T22:59:00'], [ 992271012, 47.64748, -3.509307, '2013-10-08T22:56:00'], [ 227844000, 47.13057, -3.126982, '2013-10-08T22:58:00']])","{ 227254910: [ 'LAURELINE', 'FR', ( 49.94479, -5.137455, '2013-10-08T21:51:00'), ( 49.91799, -5.315172, '2013-10-08T22:59:00')], 227844000: [ 'NEPTUNE ' 'III', 'FR', ( 47.23206, -2.913887, '2013-10-08T21:51:00'), ( 47.13057, -3.126982, '2013-10-08T22:58:00')], 255801560: [ 'AUTOPRIDE', 'PT', ( 49.3815, -4.412167, '2013-10-08T21:51:00'), ( 49.25383, -4.784833, '2013-10-08T22:59:00')], 992271012: [ 'PENMEN', 'FR', ( 47.64744, -3.509282, '2013-10-08T21:50:00'), ( 47.64748, -3.509307, '2013-10-08T22:56:00')]}","{ 227254910: [ 'LAURELINE', 'FR', ( 49.94479, -5.137455, '2013-10-08T21:51:00'), ( 49.91799, -5.315172, '2013-10-08T22:59:00')], 227844000: [ 'NEPTUNE ' 'III', 'FR', ( 47.23206, -2.913887, '2013-10-08T21:51:00'), ( 47.13057, -3.126982, '2013-10-08T22:58:00')], 255801560: [ 'AUTOPRIDE', 'PT', ( 49.3815, -4.412167, '2013-10-08T21:51:00'), ( 49.25383, -4.784833, '2013-10-08T22:59:00')], 992271012: [ 'PENMEN', 'FR', ( 47.64744, -3.509282, '2013-10-08T21:50:00'), ( 47.64748, -3.509307, '2013-10-08T22:56:00')]}",OK


### Les fichiers de données complets

Signalons enfin pour ceux qui sont intéressés que les données chargées dans cet exercice sont disponibles au format JSON - qui est précisément celui exposé par marinetraffic.

Nous avons beaucoup simplifié les données d'entrée pour vous permettre une mise au point plus facile. Si vous voulez vous amuser à charger des données un peu plus significatives, sachez que :

* vous avez accès aux fichiers de données plus complets :
  * [data/marine-e1-ext.json](data/marine-e1-ext.json)
  * [data/marine-e1-abb.json](data/marine-e1-abb.json)
* pour charger ces fichiers, qui sont donc au [format JSON](http://en.wikipedia.org/wiki/JSON), la connaissance intime de ce format n'est pas nécessaire, on peut tout simplement utiliser le [module `json`](https://docs.python.org/3/library/json.html). Voici le code utilisé dans l'exercice pour charger ces JSON en mémoire ; il utilise des notions que nous verrons dans les semaines à venir :

In [48]:
# load data from files
import json

with open("data/marine-e1-ext.json", encoding="utf-8") as feed:
    extended_full = json.load(feed)

with open("data/marine-e1-abb.json", encoding="utf-8") as feed:
    abbreviated_full = json.load(feed)

Une fois que vous avez un code qui fonctionne vous pouvez le lancer sur ces données plus copieuses en faisant :

In [49]:
exo_merge.correction(merge, extended_full, abbreviated_full)